<span style="color:red;">Create both the dependent and independent variables </span>

In [1]:
import pandas as pd
import numpy as np

<h1>All Stocks</h1>

In [3]:
"""
All 1193 stocks 
Keep only the tickers that have full observations and no 0's
Observations are from 01-2013 to 12-2022
Format: datum, metric, ticke
"""
all_stocks = pd.read_excel('z_features.xlsx', sheet_name='test_set')
all_stocks = all_stocks['Symbol']
all_stocks = set(all_stocks)
all_stocks

{'RBA',
 'TRC',
 'CCU',
 'KMB',
 'OLP',
 'PAR',
 'EDU',
 'PNW',
 'WOR',
 'ESS',
 'CRM',
 'HVT',
 'HIW',
 'NRT',
 'CCRD',
 'CIA',
 'ACRE',
 'MCK',
 'CLH',
 'SUI',
 'FNV',
 'OLN',
 'HCI',
 'TISI',
 'CUBE',
 'TK',
 'KBH',
 'AGO',
 'KMI',
 'NCR',
 'KFS',
 'SLG',
 'OGE',
 'TNC',
 'EME',
 'BFS',
 'AVB',
 'CIG',
 'CL',
 'TDC',
 'BBVA',
 'CCJ',
 'YELP',
 'ALV',
 'ING',
 'FAF',
 'BFH',
 'SLCA',
 'SHO',
 'CAG',
 'LEG',
 'CSR',
 'M',
 'TMO',
 'SMG',
 'BAC',
 'MTX',
 'CPG',
 'GPN',
 'MCO',
 'NSC',
 'FLR',
 'CYH',
 'UNP',
 'CACI',
 'XPO',
 'CNC',
 'EIX',
 'DAR',
 'MOD',
 'AEG',
 'HUM',
 'VALE',
 'KB',
 'CRI',
 'NVO',
 'G',
 'ZWS',
 'TTE',
 'UTL',
 'ORA',
 'TNP',
 'WGO',
 'INT',
 'EHC',
 'RES',
 'CEQP',
 'AZO',
 'CADE',
 'DRI',
 'EW',
 'LRN',
 'WF',
 'NNN',
 'WSO',
 'TFII',
 'JBT',
 'TECK',
 'SF',
 'GME',
 'ACR',
 'BIO',
 'ABB',
 'BHC',
 'DRQ',
 'RIG',
 'SCHW',
 'GCO',
 'CMS',
 'SCL',
 'ORAN',
 'SXT',
 'HRB',
 'FSM',
 'POR',
 'PPG',
 'CHT',
 'STNG',
 'EPAM',
 'DSX',
 'UMH',
 'AIT',
 'RGA',
 'CPK',
 

<h1>Control General</h1>

In [15]:
"""
Most controls can be made by changing the file and variable names in the code below
"""
df = pd.read_excel('controls_full_no0\data_control_price.xlsx')
df

In [37]:
df = df[df['Ticker Symbol'].isin(all_stocks)] \
    .groupby('Ticker Symbol') \
    .filter(lambda x: len(x) == 120 and not x.isna().any().any() and not (x['Price - Close - Monthly'] == 0).any()) \
    .reset_index() \
    .drop('index', axis=1) 
df

,Data Date - Security Monthly,Price - Close - Monthly,Ticker Symbol
0,2013-01-31,18.85,AIR
1,2013-02-28,17.57,AIR
2,2013-03-31,18.39,AIR
3,2013-04-30,17.86,AIR
4,2013-05-31,20.06,AIR
...,...,...,...
140875,2022-08-31,83.00,LYB
140876,2022-09-30,75.28,LYB
140877,2022-10-31,76.45,LYB
140878,2022-11-30,85.01,LYB


In [38]:
# check amount of observations
len(df)/120

1174.0

In [41]:
# df['Price - Close - Monthly'] = 1 / df['Price - Close - Monthly']
# df
df.to_excel('controls_full_no0\data_filtered_control_price.xlsx')

<h1>Control Volatility</h1>

In [42]:
"""
Here we create the monthly volatility control variable
"""
df = pd.read_csv('controls_full_no0\data_control_volatility.txt', delimiter='\t')
df = df.drop(columns=['gvkey', 'iid']).rename(columns={'datadate': 'Date', 'tic': 'Ticker Symbol', 'prccd': 'Price'})
df['Daily Returns'] = df.groupby('Ticker Symbol')['Price'].pct_change()*100
df

C:\Users\538562gz\AppData\Local\Temp\ipykernel_9528\1935930405.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('controls_full_no0\data_control_volatility.txt', delimiter='\t')


,gvkey,iid,datadate,tic,prccd
0,1004,01,2013-01-02,AIR,19.37
1,1004,01,2013-01-03,AIR,19.82
2,1004,01,2013-01-04,AIR,20.11
3,1004,01,2013-01-07,AIR,19.75
4,1004,01,2013-01-08,AIR,19.61
...,...,...,...,...,...
4386105,312009,01,2022-12-23,FRO,12.49
4386106,312009,01,2022-12-27,FRO,12.48
4386107,312009,01,2022-12-28,FRO,11.92
4386108,312009,01,2022-12-29,FRO,12.06


In [45]:
# group by both ticker and month, then calculate the volatility
df['Date'] = pd.to_datetime(df['Date']) 
monthly_volatility = df.groupby(['Ticker Symbol', pd.Grouper(key='Date', freq='M')])['Daily Returns'].std()
monthly_volatility

Ticker Symbol  Date      
0050B          2013-02-28         NaN
               2013-05-31         NaN
               2013-08-31         NaN
               2013-11-30         NaN
               2014-02-28         NaN
                               ...   
ZWS            2022-08-31    2.072921
               2022-09-30    2.119668
               2022-10-31    3.310613
               2022-11-30    2.658288
               2022-12-31    2.876602
Name: Daily Returns, Length: 212090, dtype: float64

In [46]:
df_monthly_volatility = monthly_volatility.reset_index().rename(columns={'Daily Returns': 'Monthly Volatility'})
df = df_monthly_volatility
df

,Ticker Symbol,Date,Monthly Volatility
0,0050B,2013-02-28,NaN
1,0050B,2013-05-31,NaN
2,0050B,2013-08-31,NaN
3,0050B,2013-11-30,NaN
4,0050B,2014-02-28,NaN
...,...,...,...
212085,ZWS,2022-08-31,2.072921
212086,ZWS,2022-09-30,2.119668
212087,ZWS,2022-10-31,3.310613
212088,ZWS,2022-11-30,2.658288


In [47]:
df = df[df['Ticker Symbol'].isin(all_stocks)] \
    .groupby('Ticker Symbol') \
    .filter(lambda x: len(x) == 120 and not x.isna().any().any() and not (x['Monthly Volatility'] == 0).any()) \
    .reset_index() \
    .drop('index', axis=1) 
df

,Ticker Symbol,Date,Monthly Volatility
0,A,2013-01-31,1.094107
1,A,2013-02-28,1.662541
2,A,2013-03-31,1.118800
3,A,2013-04-30,1.841820
4,A,2013-05-31,1.359981
...,...,...,...
140755,ZWS,2022-08-31,2.072921
140756,ZWS,2022-09-30,2.119668
140757,ZWS,2022-10-31,3.310613
140758,ZWS,2022-11-30,2.658288


In [48]:
len(df)/120

1173.0

In [49]:
df.to_excel('controls_full_no0\data_filtered_control_volatility.xlsx')

<h1>Control Size</h1>

In [50]:
"""
Here we create the monthly size control variable
"""
df = pd.read_excel('controls_full_no0\data_control_size.xlsx')
df['Market Capitalization'] = df['Price'] * df['Shares Outstanding Monthly']

,Data Date - Security Monthly,Ticker Symbol,Price,Shares Outstanding Monthly
0,2013-01-31,AIR,18.85,39708000.0
1,2013-02-28,AIR,17.57,39708000.0
2,2013-03-31,AIR,18.39,39733000.0
3,2013-04-30,AIR,17.86,39733000.0
4,2013-05-31,AIR,20.06,39733000.0
...,...,...,...,...
157178,2022-08-31,FRO,11.86,203531000.0
157179,2022-09-30,FRO,10.93,203531000.0
157180,2022-10-31,FRO,12.56,203531000.0
157181,2022-11-30,FRO,13.83,203531000.0


In [52]:
df = df[df['Ticker Symbol'].isin(all_stocks)] \
    .groupby('Ticker Symbol') \
    .filter(lambda x: len(x) == 120 and not x.isna().any().any() and not (x['Market Capitalization'] == 0).any()) \
    .reset_index() \
    .drop('index', axis=1) 
df

,Data Date - Security Monthly,Ticker Symbol,Price,Shares Outstanding Monthly,Market Capitalization
0,2013-01-31,AIR,18.85,39708000.0,7.484958e+08
1,2013-02-28,AIR,17.57,39708000.0,6.976696e+08
2,2013-03-31,AIR,18.39,39733000.0,7.306899e+08
3,2013-04-30,AIR,17.86,39733000.0,7.096314e+08
4,2013-05-31,AIR,20.06,39733000.0,7.970440e+08
...,...,...,...,...,...
140515,2022-08-31,LYB,83.00,326206000.0,2.707510e+10
140516,2022-09-30,LYB,75.28,326206000.0,2.455679e+10
140517,2022-10-31,LYB,76.45,325624000.0,2.489395e+10
140518,2022-11-30,LYB,85.01,325624000.0,2.768130e+10


In [53]:
len(df)/120

1171.0

In [54]:
df.to_excel('controls_full_no0\data_filtered_control_size.xlsx')

<h1>Control Illiquidity</h1>

In [ ]:
"""
Here we create the monthly illiquidity measure for each stock
It takes 3 steps:
- daily returns
- daily volume
- aggregate the daily ratios into monthly measures
""""

<h4>Daily Returns</h4>

In [71]:
df = pd.read_csv('controls_full_no0\data_control_volatility.txt', delimiter='\t')
df = df.drop(columns=['gvkey', 'iid']).rename(columns={'datadate': 'Date', 'tic': 'Ticker Symbol', 'prccd': 'Price'})
df

C:\Users\538562gz\AppData\Local\Temp\ipykernel_9528\1721254334.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('controls_full_no0\data_control_volatility.txt', delimiter='\t')


,Date,Ticker Symbol,Price
0,2013-01-02,AIR,19.37
1,2013-01-03,AIR,19.82
2,2013-01-04,AIR,20.11
3,2013-01-07,AIR,19.75
4,2013-01-08,AIR,19.61
...,...,...,...
4386105,2022-12-23,FRO,12.49
4386106,2022-12-27,FRO,12.48
4386107,2022-12-28,FRO,11.92
4386108,2022-12-29,FRO,12.06


In [72]:
df['Daily Returns'] = df.groupby('Ticker Symbol')['Price'].pct_change()*100
df['Date'] = pd.to_datetime(df['Date']) 
daily_returns = df
daily_returns

,Date,Ticker Symbol,Price,Daily Returns
0,2013-01-02,AIR,19.37,NaN
1,2013-01-03,AIR,19.82,2.323180
2,2013-01-04,AIR,20.11,1.463169
3,2013-01-07,AIR,19.75,-1.790154
4,2013-01-08,AIR,19.61,-0.708861
...,...,...,...,...
4386105,2022-12-23,FRO,12.49,0.725806
4386106,2022-12-27,FRO,12.48,-0.080064
4386107,2022-12-28,FRO,11.92,-4.487179
4386108,2022-12-29,FRO,12.06,1.174497


In [73]:
daily_returns['Daily Returns'] = daily_returns['Daily Returns'].abs()
daily_returns

,Date,Ticker Symbol,Price,Daily Returns
0,2013-01-02,AIR,19.37,NaN
1,2013-01-03,AIR,19.82,2.323180
2,2013-01-04,AIR,20.11,1.463169
3,2013-01-07,AIR,19.75,1.790154
4,2013-01-08,AIR,19.61,0.708861
...,...,...,...,...
4386105,2022-12-23,FRO,12.49,0.725806
4386106,2022-12-27,FRO,12.48,0.080064
4386107,2022-12-28,FRO,11.92,4.487179
4386108,2022-12-29,FRO,12.06,1.174497


<h4>Daily Volume</h4>

In [74]:
df = pd.read_csv('controls_full_no0\data_control_volume.txt', delimiter='\t')
df

C:\Users\538562gz\AppData\Local\Temp\ipykernel_9528\2527819357.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('controls_full_no0\data_control_volume.txt', delimiter='\t')


,gvkey,iid,datadate,tic,cshtrd
0,1004,01,2013-01-02,AIR,427126.0
1,1004,01,2013-01-03,AIR,419610.0
2,1004,01,2013-01-04,AIR,386724.0
3,1004,01,2013-01-07,AIR,303724.0
4,1004,01,2013-01-08,AIR,311604.0
...,...,...,...,...,...
4386105,312009,01,2022-12-23,FRO,3959851.0
4386106,312009,01,2022-12-27,FRO,1886300.0
4386107,312009,01,2022-12-28,FRO,3168506.0
4386108,312009,01,2022-12-29,FRO,3568007.0


In [75]:
df = df.drop(columns=['gvkey', 'iid']).rename(columns={'datadate': 'Date', 'tic': 'Ticker Symbol', 'cshtrd': 'Volume'})
df['Date'] = pd.to_datetime(df['Date']) 
daily_volume = df
daily_volume

,Date,Ticker Symbol,Volume
0,2013-01-02,AIR,427126.0
1,2013-01-03,AIR,419610.0
2,2013-01-04,AIR,386724.0
3,2013-01-07,AIR,303724.0
4,2013-01-08,AIR,311604.0
...,...,...,...
4386105,2022-12-23,FRO,3959851.0
4386106,2022-12-27,FRO,1886300.0
4386107,2022-12-28,FRO,3168506.0
4386108,2022-12-29,FRO,3568007.0


<h4>Daily Ratio to Monthly Measures</h4>

In [78]:
df = pd.merge(daily_returns, daily_volume, on=['Date', 'Ticker Symbol'])
df['Ratio'] = df['Daily Returns'] / df['Volume']
df

,Date,Ticker Symbol,Price,Daily Returns,Volume
0,2013-01-02,AIR,19.37,NaN,427126.0
1,2013-01-03,AIR,19.82,2.323180,419610.0
2,2013-01-04,AIR,20.11,1.463169,386724.0
3,2013-01-07,AIR,19.75,1.790154,303724.0
4,2013-01-08,AIR,19.61,0.708861,311604.0
...,...,...,...,...,...
4386105,2022-12-23,FRO,12.49,0.725806,3959851.0
4386106,2022-12-27,FRO,12.48,0.080064,1886300.0
4386107,2022-12-28,FRO,11.92,4.487179,3168506.0
4386108,2022-12-29,FRO,12.06,1.174497,3568007.0


In [82]:
monthly_illiquidity = df.groupby(['Ticker Symbol', pd.Grouper(key='Date', freq='M')])['Ratio'].mean()
df = monthly_illiquidity.reset_index().rename(columns={'Ratio': 'Illiquidity'})
df

Ticker Symbol  Date      
0050B          2013-02-28         NaN
               2013-05-31         NaN
               2013-08-31         NaN
               2013-11-30         NaN
               2014-02-28         NaN
                               ...   
ZWS            2022-08-31    0.000002
               2022-09-30    0.000002
               2022-10-31    0.000002
               2022-11-30    0.000002
               2022-12-31    0.000002
Name: Ratio, Length: 212090, dtype: float64

In [84]:
df = df[df['Ticker Symbol'].isin(all_stocks)] \
    .groupby('Ticker Symbol') \
    .filter(lambda x: len(x) == 120 and not x.isna().any().any() and not (x['Illiquidity'] == 0).any()) \
    .reset_index() \
    .drop('index', axis=1) 
df

,Ticker Symbol,Date,Illiquidity
0,A,2013-01-31,2.887525e-07
1,A,2013-02-28,2.961422e-07
2,A,2013-03-31,2.901643e-07
3,A,2013-04-30,3.265074e-07
4,A,2013-05-31,2.599894e-07
...,...,...,...
139555,ZWS,2022-08-31,2.262266e-06
139556,ZWS,2022-09-30,1.884943e-06
139557,ZWS,2022-10-31,1.792403e-06
139558,ZWS,2022-11-30,1.519318e-06


In [85]:
len(df)/120

1163.0

In [86]:
df.to_excel('controls_full_no0\data_filtered_control_illiquidity.xlsx')

<h1>Control Fluency</h1>

In [5]:
"""
Create for each stock its fluency variable
"""
df = pd.read_excel('z_predictions_100x.xlsx')
df

,Symbol,Name,Fluency
0,A,Agilent Technologies,622.599854
1,AAIC,Arlington Asset Investment,765.885742
2,AAP,Advance Auto Parts,1789.879639
3,AAT,American Assets Trust,1152.093750
4,AB,AllianceBernstein Holding,620.703552
...,...,...,...
1188,YELP,Yelp,1430.900146
1189,YPF,YPF,424.110168
1190,YUM,Yum Brands,2033.593994
1191,ZBH,Zimmer Biomet Holdings,424.110168


In [6]:
symbols = df['Symbol']
symbols

0          A
1       AAIC
2        AAP
3        AAT
4         AB
        ... 
1188    YELP
1189     YPF
1190     YUM
1191     ZBH
1192     ZWS
Name: Symbol, Length: 1193, dtype: object

In [7]:
fluency = df['Fluency']
fluency

0        622.599854
1        765.885742
2       1789.879639
3       1152.093750
4        620.703552
           ...     
1188    1430.900146
1189     424.110168
1190    2033.593994
1191     424.110168
1192     424.110168
Name: Fluency, Length: 1193, dtype: float64

In [8]:
repeated_symbols = []
for symbol in symbols:
    repeated_symbols = repeated_symbols + [symbol]*120
    
repeated_symbols

['A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 'AAIC',
 

In [9]:
repeated_fluency = []
for score in fluency:
    repeated_fluency = repeated_fluency + [score]*120
    
repeated_fluency

[622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.599853515625,
 622.5998535

In [12]:
len(repeated_fluency), type(repeated_fluency)

(143160, list)

In [13]:
dates = pd.date_range('2013-01-01','2022-12-31', freq='M').tolist()
repeated_dates = dates*1193
repeated_dates

[Timestamp('2013-01-31 00:00:00', freq='M'),
 Timestamp('2013-02-28 00:00:00', freq='M'),
 Timestamp('2013-03-31 00:00:00', freq='M'),
 Timestamp('2013-04-30 00:00:00', freq='M'),
 Timestamp('2013-05-31 00:00:00', freq='M'),
 Timestamp('2013-06-30 00:00:00', freq='M'),
 Timestamp('2013-07-31 00:00:00', freq='M'),
 Timestamp('2013-08-31 00:00:00', freq='M'),
 Timestamp('2013-09-30 00:00:00', freq='M'),
 Timestamp('2013-10-31 00:00:00', freq='M'),
 Timestamp('2013-11-30 00:00:00', freq='M'),
 Timestamp('2013-12-31 00:00:00', freq='M'),
 Timestamp('2014-01-31 00:00:00', freq='M'),
 Timestamp('2014-02-28 00:00:00', freq='M'),
 Timestamp('2014-03-31 00:00:00', freq='M'),
 Timestamp('2014-04-30 00:00:00', freq='M'),
 Timestamp('2014-05-31 00:00:00', freq='M'),
 Timestamp('2014-06-30 00:00:00', freq='M'),
 Timestamp('2014-07-31 00:00:00', freq='M'),
 Timestamp('2014-08-31 00:00:00', freq='M'),
 Timestamp('2014-09-30 00:00:00', freq='M'),
 Timestamp('2014-10-31 00:00:00', freq='M'),
 Timestamp

In [14]:
len(repeated_dates)

143160

In [15]:
repeated_df = pd.DataFrame({'Ticker Symbol': repeated_symbols, 'Date': repeated_dates, 'Fluency': repeated_fluency})
repeated_df

,Ticker Symbol,Date,Fluency
0,A,2013-01-31,622.599854
1,A,2013-02-28,622.599854
2,A,2013-03-31,622.599854
3,A,2013-04-30,622.599854
4,A,2013-05-31,622.599854
...,...,...,...
143155,ZWS,2022-08-31,424.110168
143156,ZWS,2022-09-30,424.110168
143157,ZWS,2022-10-31,424.110168
143158,ZWS,2022-11-30,424.110168


In [16]:
repeated_df.to_excel('controls_full_no0\data_filtered_control_fluency.xlsx')

<h1>Merging Controls</h1>

In [17]:
"""
Merge all control dfs into one big df, which is used in a regression
This is done for each characteristic
"""

'\nMerge all control dfs into one big df, which is used in a regression\nThis is done for each characteristic\n'

In [18]:
book_to_market = pd.read_excel('controls_full_no0\data_filtered_control_bm.xlsx')
book_to_market

,Date,Book/Market,Ticker Symbol
0,2013-01-31,0.542476,AMRC
1,2013-02-28,0.646842,AMRC
2,2013-03-31,0.646842,AMRC
3,2013-04-30,0.646842,AMRC
4,2013-05-31,0.854068,AMRC
...,...,...,...
74755,2022-08-31,0.732472,VPG
74756,2022-09-30,0.732472,VPG
74757,2022-10-31,0.732472,VPG
74758,2022-11-30,0.724565,VPG


In [19]:
ps = pd.read_excel('controls_full_no0\data_filtered_control_ps.xlsx')
ps

,Date,Price/Sales,Ticker Symbol
0,2013-01-31,4.515576,ORCL
1,2013-02-28,4.340010,ORCL
2,2013-03-31,4.090668,ORCL
3,2013-04-30,4.145753,ORCL
4,2013-05-31,4.224659,ORCL
...,...,...,...
81595,2022-08-31,1.244434,VPG
81596,2022-09-30,1.073238,VPG
81597,2022-10-31,1.224122,VPG
81598,2022-11-30,1.428888,VPG


In [20]:
profitability = pd.read_excel('controls_full_no0\data_filtered_control_profitability.xlsx')
profitability

,Date,Gross Profit/Total Assets,Ticker Symbol
0,2013-01-31,0.391929,ORCL
1,2013-02-28,0.391929,ORCL
2,2013-03-31,0.391929,ORCL
3,2013-04-30,0.387195,ORCL
4,2013-05-31,0.387195,ORCL
...,...,...,...
81475,2022-08-31,0.352589,VPG
81476,2022-09-30,0.352589,VPG
81477,2022-10-31,0.352589,VPG
81478,2022-11-30,0.361082,VPG


In [22]:
asset_turnover = pd.read_excel('controls_full_no0\data_filtered_control_asset_turnover.xlsx')
asset_turnover

,Date,Asset Turnover,Ticker Symbol
0,2013-01-31,0.482093,ORCL
1,2013-02-28,0.482093,ORCL
2,2013-03-31,0.482093,ORCL
3,2013-04-30,0.473248,ORCL
4,2013-05-31,0.473248,ORCL
...,...,...,...
80755,2022-08-31,0.762084,VPG
80756,2022-09-30,0.762084,VPG
80757,2022-10-31,0.762084,VPG
80758,2022-11-30,0.777952,VPG


In [23]:
rd = pd.read_excel('controls_full_no0\data_filtered_control_rd.xlsx')
rd

,Date,Research and Development/Sales,Ticker Symbol
0,2013-01-31,0.128149,ORCL
1,2013-02-28,0.128149,ORCL
2,2013-03-31,0.128149,ORCL
3,2013-04-30,0.129532,ORCL
4,2013-05-31,0.129532,ORCL
...,...,...,...
27955,2022-08-31,0.152131,CALX
27956,2022-09-30,0.152131,CALX
27957,2022-10-31,0.152131,CALX
27958,2022-11-30,0.149277,CALX


In [24]:
advertising = pd.read_excel('controls_full_no0\data_filtered_control_advertising.xlsx')
advertising

,Date,Avertising Expenses/Sales,Ticker Symbol
0,2013-01-31,0.002128,ORCL
1,2013-02-28,0.002128,ORCL
2,2013-03-31,0.002128,ORCL
3,2013-04-30,0.002128,ORCL
4,2013-05-31,0.002128,ORCL
...,...,...,...
28075,2022-08-31,0.037243,SIX
28076,2022-09-30,0.037243,SIX
28077,2022-10-31,0.037243,SIX
28078,2022-11-30,0.037243,SIX


In [25]:
leverage = pd.read_excel('controls_full_no0\data_filtered_control_leverage.xlsx')
leverage

,Date,Total Debt/Total Assets,Ticker Symbol
0,2013-01-31,0.212944,ORCL
1,2013-02-28,0.212944,ORCL
2,2013-03-31,0.212944,ORCL
3,2013-04-30,0.225337,ORCL
4,2013-05-31,0.225337,ORCL
...,...,...,...
73315,2022-08-31,0.194819,VPG
73316,2022-09-30,0.194819,VPG
73317,2022-10-31,0.194819,VPG
73318,2022-11-30,0.192046,VPG


In [26]:
size = pd.read_excel('controls_full_no0\data_filtered_control_size.xlsx')
size

,Ticker Symbol,Date,Market Capitalization
0,AIR,2013-01-31,7.484958e+08
1,AIR,2013-02-28,6.976696e+08
2,AIR,2013-03-31,7.306899e+08
3,AIR,2013-04-30,7.096314e+08
4,AIR,2013-05-31,7.970440e+08
...,...,...,...
140515,LYB,2022-08-31,2.707510e+10
140516,LYB,2022-09-30,2.455679e+10
140517,LYB,2022-10-31,2.489395e+10
140518,LYB,2022-11-30,2.768130e+10


In [27]:
price = pd.read_excel('controls_full_no0\data_filtered_control_price.xlsx')
price

,Date,Price - Close - Monthly,Ticker Symbol
0,2013-01-31,0.053050,AIR
1,2013-02-28,0.056915,AIR
2,2013-03-31,0.054377,AIR
3,2013-04-30,0.055991,AIR
4,2013-05-31,0.049850,AIR
...,...,...,...
140875,2022-08-31,0.012048,LYB
140876,2022-09-30,0.013284,LYB
140877,2022-10-31,0.013080,LYB
140878,2022-11-30,0.011763,LYB


In [28]:
volatility = pd.read_excel('controls_full_no0\data_filtered_control_volatility.xlsx')
volatility

,Ticker Symbol,Date,Monthly Volatility
0,A,2013-01-31,1.094107
1,A,2013-02-28,1.662541
2,A,2013-03-31,1.118800
3,A,2013-04-30,1.841820
4,A,2013-05-31,1.359981
...,...,...,...
140755,ZWS,2022-08-31,2.072921
140756,ZWS,2022-09-30,2.119668
140757,ZWS,2022-10-31,3.310613
140758,ZWS,2022-11-30,2.658288


In [29]:
fluency = pd.read_excel('controls_full_no0\data_filtered_control_fluency.xlsx')
fluency

,Ticker Symbol,Date,Fluency
0,A,2013-01-31,622.599854
1,A,2013-02-28,622.599854
2,A,2013-03-31,622.599854
3,A,2013-04-30,622.599854
4,A,2013-05-31,622.599854
...,...,...,...
143155,ZWS,2022-08-31,424.110168
143156,ZWS,2022-09-30,424.110168
143157,ZWS,2022-10-31,424.110168
143158,ZWS,2022-11-30,424.110168


In [30]:
illiquidity = pd.read_excel('controls_full_no0/data_filtered_control_illiquidity.xlsx')
illiquidity

,Ticker Symbol,Date,Illiquidity
0,A,2013-01-31,2.887525e-07
1,A,2013-02-28,2.961422e-07
2,A,2013-03-31,2.901643e-07
3,A,2013-04-30,3.265074e-07
4,A,2013-05-31,2.599894e-07
...,...,...,...
139555,ZWS,2022-08-31,2.262266e-06
139556,ZWS,2022-09-30,1.884943e-06
139557,ZWS,2022-10-31,1.792403e-06
139558,ZWS,2022-11-30,1.519318e-06


In [31]:
returns = pd.read_excel('controls_full_no0/data_filtered_control_return.xlsx')
returns

,Ticker Symbol,Date,Monthly Total Return
0,AIR,2013-01-31,1.311563
1,AIR,2013-02-28,-6.790451
2,AIR,2013-03-31,4.667046
3,AIR,2013-04-30,-2.474171
4,AIR,2013-05-31,12.318029
...,...,...,...
114115,LYB,2022-08-31,-5.531867
114116,LYB,2022-09-30,-9.301205
114117,LYB,2022-10-31,1.554198
114118,LYB,2022-11-30,12.753434


<h4>Performance</h4>

In [32]:
# merging for performance
results_performance = pd.merge(returns, fluency, on=['Date', 'Ticker Symbol'])

In [33]:
len(results_performance)/120

951.0

In [34]:
results_performance

,Ticker Symbol,Date,Monthly Total Return,Fluency
0,AIR,2013-01-31,1.311563,424.110168
1,AIR,2013-02-28,-6.790451,424.110168
2,AIR,2013-03-31,4.667046,424.110168
3,AIR,2013-04-30,-2.474171,424.110168
4,AIR,2013-05-31,12.318029,424.110168
...,...,...,...,...
114115,LYB,2022-08-31,-5.531867,424.110168
114116,LYB,2022-09-30,-9.301205,424.110168
114117,LYB,2022-10-31,1.554198,424.110168
114118,LYB,2022-11-30,12.753434,424.110168


In [35]:
results_performance.to_excel('controls_full_no0/data_reg_performance.xlsx')

<h4>Illiquidity</h4>

In [36]:
results_illiquidity = pd.merge(illiquidity, fluency, on=['Date', 'Ticker Symbol'])
results_illiquidity = pd.merge(results_illiquidity, size, on=['Date', 'Ticker Symbol'])
results_illiquidity = pd.merge(results_illiquidity, profitability, on=['Date', 'Ticker Symbol'])
results_illiquidity = pd.merge(results_illiquidity, book_to_market, on=['Date', 'Ticker Symbol'])
results_illiquidity = pd.merge(results_illiquidity, advertising, on=['Date', 'Ticker Symbol'])
results_illiquidity = pd.merge(results_illiquidity, price, on=['Date', 'Ticker Symbol'])
results_illiquidity = pd.merge(results_illiquidity, volatility, on=['Date', 'Ticker Symbol'])

len(results_illiquidity)/120

202.0

In [37]:
results_illiquidity

,Ticker Symbol,Date,Illiquidity,Fluency,Market Capitalization,Gross Profit/Total Assets,Book/Market,Avertising Expenses/Sales,Price - Close - Monthly,Monthly Volatility
0,A,2013-01-31,2.887525e-07,622.599854,1.558053e+10,0.376044,0.431852,0.007291,0.022331,1.094107
1,A,2013-02-28,2.961422e-07,622.599854,1.439223e+10,0.376044,0.431852,0.007291,0.024108,1.662541
2,A,2013-03-31,2.901643e-07,622.599854,1.457026e+10,0.396045,0.344016,0.007291,0.023827,1.118800
3,A,2013-04-30,3.265074e-07,622.599854,1.438627e+10,0.396045,0.344016,0.007291,0.024131,1.841820
4,A,2013-05-31,2.599894e-07,622.599854,1.577838e+10,0.396045,0.344016,0.007291,0.022002,1.359981
...,...,...,...,...,...,...,...,...,...,...
24235,YELP,2022-08-31,2.562980e-06,1430.900146,2.401263e+09,0.974274,0.363855,0.054563,0.029265,4.356689
24236,YELP,2022-09-30,2.407408e-06,1430.900146,2.382991e+09,0.974274,0.363855,0.054563,0.029490,1.981414
24237,YELP,2022-10-31,2.303998e-06,1430.900146,2.699224e+09,0.974274,0.363855,0.054563,0.026035,1.812760
24238,YELP,2022-11-30,2.498737e-06,1430.900146,2.157277e+09,1.018609,0.293804,0.054563,0.032310,4.234128


In [38]:
results_illiquidity.to_excel('controls_full_no0/data_reg_illiquidity.xlsx')

<h4>Valuation</h4>

In [39]:
results_valuation = pd.merge(book_to_market, fluency, on=['Date', 'Ticker Symbol'])
results_valuation = pd.merge(results_valuation, ps, on=['Date', 'Ticker Symbol'])
results_valuation = pd.merge(results_valuation, profitability, on=['Date', 'Ticker Symbol'])
results_valuation = pd.merge(results_valuation, leverage, on=['Date', 'Ticker Symbol'])
results_valuation = pd.merge(results_valuation, advertising, on=['Date', 'Ticker Symbol'])

len(results_valuation)/120

167.0

In [40]:
results_valuation

,Date,Book/Market,Ticker Symbol,Fluency,Price/Sales,Gross Profit/Total Assets,Total Debt/Total Assets,Avertising Expenses/Sales
0,2013-01-31,0.133712,BC,622.776367,0.880432,0.399588,0.263404,0.009578
1,2013-02-28,0.065353,BC,622.776367,0.883946,0.426532,0.235872,0.009280
2,2013-03-31,0.065353,BC,622.776367,0.832497,0.426532,0.235872,0.009280
3,2013-04-30,0.065353,BC,622.776367,0.770218,0.426532,0.235872,0.009280
4,2013-05-31,0.073331,BC,622.776367,0.809050,0.416177,0.242719,0.009280
...,...,...,...,...,...,...,...,...
20035,2022-08-31,0.006457,EXPR,1198.341064,0.071124,0.533481,0.715633,0.072181
20036,2022-09-30,0.006457,EXPR,1198.341064,0.055062,0.544412,0.706854,0.072181
20037,2022-10-31,0.006457,EXPR,1198.341064,0.052945,0.544412,0.706854,0.072181
20038,2022-11-30,0.006457,EXPR,1198.341064,0.049571,0.544412,0.706854,0.072181


In [41]:
results_valuation.to_excel('controls_full_no0/data_reg_valuation.xlsx')